### Implementing MPC with perfect forecast in CityLearn

Import packages

In [1]:
# System operations
import inspect
import os
import uuid

# Date and time
from datetime import datetime

# type hinting
from typing import Any, List, Mapping, Tuple, Union

# Data visualization
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

# User interaction
from IPython.display import clear_output
from ipywidgets import Button, FloatSlider, HBox, HTML
from ipywidgets import IntProgress, Text, VBox

# Data manipulation
from bs4 import BeautifulSoup
import math
import numpy as np
import pandas as pd
import random
import re
import requests
import simplejson as json

# CityLearn
from citylearn.agents.rbc import HourRBC
from citylearn.agents.q_learning import TabularQLearning
from citylearn.citylearn import CityLearnEnv
from citylearn.data import DataSet
from citylearn.reward_function import RewardFunction
from citylearn.wrappers import NormalizedObservationWrapper
from citylearn.wrappers import StableBaselines3Wrapper
from citylearn.wrappers import TabularQLearningWrapper

# baseline RL algorithms
from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import BaseCallback

# helper functions to parse KPIs 
from kpi_utils import get_kpis, plot_building_kpis, plot_district_kpis
from kpi_utils import plot_building_load_profiles, plot_district_load_profiles, plot_battery_soc_profiles, plot_simulation_summary

# packages for MPC
from scipy.optimize import minimize
import cvxpy as cp

Load data from CityLearn 2021 Challenge. 

In this challenge, the agents must control the energy stored by a micro-grid of 9 buildings in real-time for a period of 4 simulated years, on an hourly time-scale. Each building has 3 action-variables (domestic hot water storage, chilled water storage, and electrical storage), except for buildings 3 and 4 that do not have domestic hot water storage.

In [2]:
DATASET_NAME = 'citylearn_challenge_2021'
ds = DataSet()
schema = ds.get_schema(DATASET_NAME)

Data Preprocessing:
1. Use only 3, 5, 7, or 9 buildings 
2. Use the first two years of data for training, the third year for validation, and the fourth year for testing.
3. Use centralized control

These can be modified directly in the schema. The buildings will be pseudo-randomly selected with a seed for reproducibility. 

In [3]:
from env_utils import get_n_buildings, set_schema_buildings, set_schema_simulation_period

# select the number of buildings in simulation
scale = 'tiny'
n_buildings = get_n_buildings(scale)

 # select the number of days in simulation
split = 'train'

schema, buildings = set_schema_buildings(schema, scale)
schema, simulation_start_time_step, simulation_end_time_step =\
    set_schema_simulation_period(schema, split)

print('Number of buildings:', n_buildings)
print('Selected buildings:', buildings)
print(
    f'Selected {split} split:',
    (simulation_start_time_step, simulation_end_time_step)
)

# use centralized control
schema['central_agent'] = True

Number of buildings: 3
Selected buildings: ['Building_2', 'Building_6', 'Building_9']
Selected train split: (0, 17520)


Initialize a CityLearn Environment

In [4]:
env = CityLearnEnv(schema)

# check initialization
print('Current time step:', env.time_step)
print('environment number of time steps:', env.time_steps)
print('environment uses central agent:', env.central_agent)
print('Number of buildings:', len(env.buildings))
print('Electrical storage capacity:', {
    b.name: b.electrical_storage.capacity for b in env.buildings
})
print('Electrical storage nominal power:', {
    b.name: b.electrical_storage.nominal_power for b in env.buildings
})

Current time step: 0
environment number of time steps: 17521
environment uses central agent: True
Number of buildings: 3
Electrical storage capacity: {'Building_2': 80.0, 'Building_6': 30.0, 'Building_9': 35.0}
Electrical storage nominal power: {'Building_2': 40.0, 'Building_6': 10.0, 'Building_9': 20.0}


Given a set of GEBs, the controllers aim to minimize the electricity cost, average peak demand, and carbon intensity of electricity consumed by the buildings. The evalu
tion will be based on these metrics while also including computational costs suchas 
 training time, inference time, RAM usage, and scalability across different numbe off
 builgs.

Three built-in key performance indicators can be used: cost, carbon emissions, and average daily peak. Average daily peak is a district-level KPI that is calculated using the aggregated district-level hourly net electricity consumption (kWh), $E_h^{\textrm{district}}$. Cost and carbon emissions are building-level KPIs that are calculated using the building-level hourly net electricity consumption (kWh), $E_h^{\textrm{building}}$, and are reported at the grid level as the average of the building-level values.

Cost is defined as the sum of building-level imported electricity cost, $E_h^{\textrm{building}} \times T_h$ \\$), where $T_h$ is the electricity rate at hour $h$.

$$
    \textrm{cost} = \sum_{h=0}^{n-1}{\textrm{max} \left (0,E_h^{\textrm{building}} \times T_h \right )}
$$

Carbon emissions is the sum of building-level carbon emissions (kg<sub>CO<sub>2</sub>e</sub>), $E_h^{\textrm{building}} \times O_h$, where $O_h$ is the carbon intensity (kg<sub>CO<sub>2</sub>e</sub>/kWh) at hour $h$.

$$
    \textrm{carbon emissions} = \sum_{h=0}^{n-1}{\textrm{max} \left (0,E_h^{\textrm{building}} \times O_h \right )}
$$

Average daily peak, is defined as the mean of the daily $E_h^{\textrm{district}}$ peak where $d$ is the day index and $n$ is the total number of days.

$$
    \textrm{average daily peak} = \frac{
        {\sum}_{d=0}^{n - 1} {\sum}_{h=0}^{23} {\textrm{max} \left (E_{24d + h}^{\textrm{district}}, \dots, E_{24d + 23}^{\textrm{district}} \right)}
    }{n}
$$

The KPIs are reported as normalized values with respect to the baseline outcome where the baseline outcome is when buildings are not equipped with batteries i.e., no control.

$$\textrm{KPI} = \frac{{\textrm{KPI}_{control}}}{\textrm{KPI}_{baseline (no\ battery)}}
$$

#### Implementing MPC 

In [5]:
# Define MPC optimization function
def mpc_optimization(electrical_load, cooling_load, dhw_load,
                     pv, price, carbon_intensity, battery, N):
    """
    Computes the optimal battery action over an N-hour control horizon to minimize cost.
    
    Parameters:
    - electrical_load (array): Electricity demand over the next N hours.
    - cooling_load (array): Cooling demand over the next N hours.
    - dhw_load (array): Domestic hot water demand over the next N hours.
    - pv (array): Solar generation over the next N hours.
    - price (array): Electricity price per kWh over the next N hours.
    - carbon_intensity (array): Carbon intensity per kWh over the next N hours.
    - battery (citylearn.energy_model.Battery): Battery model.
    - N (int): Control horizon.

    Returns:
    - First action in the optimal sequence.
    """
    # decision variables 
    battery_action = cp.Variable(N)
    cooling_action = cp.Variable(N)
    dhw_action = cp.Variable(N)

    # constraints
    constraints = [cp.abs(battery_action) <= 1,
                   cp.abs(cooling_action) <= 1,
                   cp.abs(dhw_action) <= 1]

    # objective
    net_load = electrical_load + cooling_load + dhw_load - pv + battery_action*battery.capacity
    dollar = cp.sum(cp.multiply(net_load, price))
    emission = cp.sum(cp.multiply(net_load, carbon_intensity))
    objective = cp.Minimize(2*dollar + 1*emission) # weights from https://www.e3s-conferences.org/articles/e3sconf/abs/2023/33/e3sconf_iaqvec2023_04018/e3sconf_iaqvec2023_04018.html

    # solve 
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.ECOS)

    return [battery_action.value[0] if battery_action.value is not None else 0.0,
            dhw_action.value[0] if dhw_action.value is not None else 0.0,
            cooling_action.value[0] if cooling_action.value is not None else 0.0]

In [ ]:
# MPC control loop with perfect forecasts
electricity_pricing_ind = np.where(np.array(list(schema['observations'].keys())) == 'electricity_pricing')[0][0].item()
carbon_intensity_ind = np.where(np.array(list(schema['observations'].keys())) == 'carbon_intensity')[0][0].item()
hour_ind = np.where(np.array(list(schema['observations'].keys())) == 'hour')[0][0].item()

obs, actions, rewards, terminated, truncated = env.reset(), [], [], False, False
obs, info = obs

while not (terminated or truncated):
    for b in env.buildings:
        # Extract relevant features from observations
        electricity_price = obs[0][electricity_pricing_ind]
        carbon_intensity = obs[0][carbon_intensity_ind]
        hour = obs[0][hour_ind]
        
        # Get perfect forecast for next N hours
        N_forecast = 12
        steps_remaining = env.time_steps - env.time_step
        N = min(N_forecast, steps_remaining)
        
        electrical_load = b.energy_simulation.non_shiftable_load[env.time_step : env.time_step+N]
        cooling_load = b.energy_simulation.cooling_demand[env.time_step : env.time_step+N]
        dhw_load = b.energy_simulation.dhw_demand[env.time_step : env.time_step+N]
        pv = b.energy_simulation.solar_generation[env.time_step : env.time_step+N]
        price = b.pricing.electricity_pricing[env.time_step : env.time_step+N]
        carbon_intensity = b.carbon_intensity.carbon_intensity[env.time_step : env.time_step+N]
        
        
        # Select action
        action = mpc_optimization(electrical_load, cooling_load, dhw_load,
                                  pv, price, carbon_intensity, 
                                  b.electrical_storage, N)
        actions.extend(action)
    obs, reward, terminated, truncated, info = env.step([actions])
    actions = []
    rewards.append(reward[0].item())

In [15]:
print(f"Total reward: {sum(rewards)}")

Total reward: -1013058.3095998764
